# Purpose
The purpose of this notebook is to separate the potential operators by taking advantage of the linearity of the SMS chiral semi-regulated momentum space potential in the coupling constants for all partial waves.

And then to load the individual matrix elements in an HDF5 file for external use.

# Notebook Setup

## Library import
We import all the required Python libraries

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import h5py
import numpy as np
import pandas as pd

## Local library import
We import all the required local libraries

In [6]:
from constants import *

from mesh import BuildMesh
from sms_chiral_potential import SMSChiralPot

# Parameters

In [7]:
### MESH INPUT PARAMETERS ###
ki = 0
cut_reg1 = 4 # Mesh cutoff of first region
cut_reg2 = 8 # Mesh cutoff of second region
kf = 100

pts_reg1 = 40 # Total points in first region
pts_reg2 = 20 # Total points in second region
pts_reg3 = 20 # Total points in third region
Ntot = pts_reg1 + pts_reg2 + pts_reg3

### POTENTIAL INPUT PARAMETERS ###
force = 'np' # Choice of interaction
ostat = 5 # Order of EFT
cutnum = 2 # Choice of cutoff
jmax = 20 # Momentum

# Creating h5 file

In [8]:
# Nodes/weights
mesh_nodes = [ki, cut_reg1, cut_reg2, kf]
pts_per_region = [pts_reg1, pts_reg2, pts_reg3]
mesh = BuildMesh(mesh_nodes, pts_per_region)
nodes, weights = mesh.nodes, mesh.weights

In [9]:
# open a new h5 file for writing
# initialize the h5 file

file_name = './potential_SMS_n4lo_plus_' + force + \
            '_Lambda450MeV_jmax-'+ str(jmax) + '_' + str(ki) \
            + str(cut_reg1) + str(cut_reg2) + '_' + str(pts_reg1) \
            + str(pts_reg2) + str(pts_reg3) + '.h5'
hf_sms = h5py.File(file_name, 'w')

In [10]:
# hard-wired numbers temporarily --> do better!
num_lecs = 30    # includes through F-wave contacts
num_sing_pws = 42
num_coup_pws = 20

num_k = len(nodes)
# V0 is the part of the potential independent of the (non-\pi N) LECs
V0_sing = np.zeros((num_sing_pws, num_k, num_k))
hf_sms.create_dataset('V0_sing', data=V0_sing)
print('shape of V0_sing (uncoupled potentials): ',V0_sing.shape)

V0_coup = np.zeros((num_coup_pws, 2*num_k, 2*num_k))
hf_sms.create_dataset('V0_coup', data=V0_coup)
print('shape of V0_coup (coupled potentials): ', V0_coup.shape)

# Each Vi corresponds to one of the (non-\pi N) LECs
Vi_sing = np.zeros((num_sing_pws, num_k, num_k, num_lecs))
hf_sms.create_dataset('Vi_sing', data=Vi_sing)
print('shape of Vi_sing (uncoupled potentials): ',Vi_sing.shape)

Vi_coup = np.zeros((num_coup_pws, 2*num_k, 2*num_k, num_lecs))
hf_sms.create_dataset('Vi_coup', data=Vi_coup)
print('shape of Vi_coup (coupled potentials): ', Vi_coup.shape)

hf_sms.create_dataset('k', data=nodes)
hf_sms.create_dataset('dk', data=weights)

quadratic = np.zeros(num_lecs, dtype=bool)
hf_sms.create_dataset('quadratic', data=quadratic)

shape of V0_sing (uncoupled potentials):  (42, 80, 80)
shape of V0_coup (coupled potentials):  (20, 160, 160)
shape of Vi_sing (uncoupled potentials):  (42, 80, 80, 30)
shape of Vi_coup (coupled potentials):  (20, 160, 160, 30)


<HDF5 dataset "quadratic": shape (30,), type "|b1">

In [11]:
dt = h5py.special_dtype(vlen=str) # data type for the LEC name strings

LEC_names = np.array(['LO 1S0 pp', 'LO 1S0 np', 'LO 1S0 nn', 'LO 3S1 np', 
                      'NLO 1S0', 'NLO 3P0', 'NLO 1P1', 'NLO 3P1', 'NLO 3S1', 
                      'NLO 3S1-3D1', 'NLO 3P2', 'N3LO t1S0', 'N3LO 1S0', 
                      'N3LO 3P0', 'N3LO 1P1', 'N3LO 3P1', 'N3LO t3S1', 
                      'N3LO 3S1', 'N3LO 3D1', 'N3LO t3S1-3D1', 'N3LO 3S1-3D1', 
                      'N3LO 1D2', 'N3LO 3D2', 'N3LO 3P2', 'N3LO 3P2-3F2', 
                      'N3LO 3D3', 'N4LO+ 3F2', 'N4LO+ 1F3', 'N4LO+ 3F3', 'N4LO+ 3F4'], 
                      dtype=dt)

hf_sms.create_dataset('lec names', data=LEC_names)  # add to the .h5 file

<HDF5 dataset "lec names": shape (30,), type "|O">

In [12]:
single = np.array([[0, 0, 0, 1], [1, 1, 0, 1], [1, 0, 1, 0], [1, 1, 1, 1], 
                    [2, 0, 2, 1], [2, 1, 2, 0], [3, 0, 3, 0], [3, 1, 3, 1], 
                    [4, 0, 4, 1], [4, 1, 4, 0], [5, 0, 5, 0], [5, 1, 5, 1], 
                    [6, 0, 6, 1], [6, 1, 6, 0], [7, 0, 7, 0], [7, 1, 7, 1], 
                    [8, 0, 8, 1], [8, 1, 8, 0], [9, 0, 9, 0], [9, 1, 9, 1], 
                    [10, 0, 10, 1], [10, 1, 10, 0], [11, 0, 11, 0], [11, 1, 11, 1], 
                    [12, 0, 12, 1], [12, 1, 12, 0], [13, 0, 13, 0], [13, 1, 13, 1], 
                    [14, 0, 14, 1], [14, 1, 14, 0], [15, 0, 15, 0], [15, 1, 15, 1], 
                    [16, 0, 16, 1], [16, 1, 16, 0], [17, 0, 17, 0], [17, 1, 17, 1], 
                    [18, 0, 18, 1], [18, 1, 18, 0], [19, 0, 19, 0], [19, 1, 19, 1], 
                    [20, 0, 20, 1], [20, 1, 20, 0]])

coupled = np.array([[[0, 1, 1, 0], [2, 1, 1, 0]], [[1, 1, 2, 1], [3, 1, 2, 1]], 
                    [[2, 1, 3, 0], [4, 1, 3, 0]], [[3, 1, 4, 1], [5, 1, 4, 1]], 
                    [[4, 1, 5, 0], [6, 1, 5, 0]], [[5, 1, 6, 1], [7, 1, 6, 1]], 
                    [[6, 1, 7, 0], [8, 1, 7, 0]], [[7, 1, 8, 1], [9, 1, 8, 1]], 
                    [[8, 1, 9, 0], [10, 1, 9, 0]], [[9, 1, 10, 1], [11, 1, 10, 1]], 
                    [[10, 1, 11, 0], [12, 1, 11, 0]], [[11, 1, 12, 1], [13, 1, 12, 1]], 
                    [[12, 1, 13, 0], [14, 1, 13, 0]], [[13, 1, 14, 1], [15, 1, 14, 1]], 
                    [[14, 1, 15, 0], [16, 1, 15, 0]], [[15, 1, 16, 1], [17, 1, 16, 1]], 
                    [[16, 1, 17, 0], [18, 1, 17, 0]], [[17, 1, 18, 1], [19, 1, 18, 1]], 
                    [[18, 1, 19, 0], [20, 1, 19, 0]], [[19, 1, 20, 1], [21, 1, 20, 1]]])

hf_sms.create_dataset('waves_coup', data=coupled)  # add to the .h5 file
hf_sms.create_dataset('waves_sing', data=single)  # add to the .h5 file

<HDF5 dataset "waves_sing": shape (42, 4), type "<i4">

In [13]:
list(hf_sms.keys())  # check the keys

['V0_coup',
 'V0_sing',
 'Vi_coup',
 'Vi_sing',
 'dk',
 'k',
 'lec names',
 'quadratic',
 'waves_coup',
 'waves_sing']

In [14]:
%%time
my_sms = SMSChiralPot(ostat, force, cutnum)
cc_pred = my_sms.get_LECs()
nodes_GeV, weights_GeV = hbar_c_GeV*nodes, hbar_c_GeV*weights
spectral, contacts = my_sms.get_smschiral(nodes_GeV, weights_GeV, jmax)

Semilocal momentum-space chiral NN potential at N4LO [Q^5] + N5LO [Q^6]
Contacts in 3F2, 1F3, 3F3, 3F4
Cutoff value: lambda = 450 MeV


jmom = 0
jmom = 1
jmom = 2
jmom = 3
jmom = 4
jmom = 5
jmom = 6
jmom = 7
jmom = 8
jmom = 9
jmom = 10
jmom = 11
jmom = 12
jmom = 13
jmom = 14
jmom = 15
jmom = 16
jmom = 17
jmom = 18
jmom = 19
jmom = 20
Wall time: 18min 31s


In [15]:
# Fill up the V0 entries

for j_index in range(0, jmax + 1):
    pot_spec = (V_factor_RME / (0.5*np.pi)) * spectral[j_index,:,:,:]
    hf_sms['V0_sing'][2*j_index,:,:] = pot_spec[0,:,:]
        
    if j_index == 0:
        hf_sms['V0_sing'][2*j_index+1,:,:] = pot_spec[5,:,:]
        
    elif j_index >= 1:
        hf_sms['V0_sing'][2*j_index+1,:,:] = pot_spec[1,:,:]

        hf_sms['V0_coup'][j_index-1,:num_k,:num_k] = pot_spec[2,:,:]
        hf_sms['V0_coup'][j_index-1,:num_k,num_k:2*num_k] = pot_spec[3,:,:]
        hf_sms['V0_coup'][j_index-1,num_k:2*num_k,:num_k] = pot_spec[4,:,:]
        hf_sms['V0_coup'][j_index-1,num_k:2*num_k,num_k:2*num_k] = pot_spec[5,:,:]


In [16]:
# Fill up the Vi entries

for j_index in range(jmax + 1):
    pot_op = (V_factor_RME / (0.5*np.pi)) * contacts[j_index]

    if (j_index == 0):
        hf_sms['Vi_sing'][2*j_index,:num_k,:num_k,1] = pot_op[0] # 'CT_1S0'
        hf_sms['Vi_sing'][2*j_index,:num_k,:num_k,4] = pot_op[1] # 'C_1S0'
        hf_sms['Vi_sing'][2*j_index,:num_k,:num_k,12] = pot_op[2] # 'D_1S0'        
        hf_sms['Vi_sing'][2*j_index+1,:num_k,:num_k,5] = pot_op[3] # 'C_3P0'
        hf_sms['Vi_sing'][2*j_index+1,:num_k,:num_k,13] = pot_op[4] # 'D_3P0'

    elif (j_index == 1):    
        hf_sms['Vi_sing'][2*j_index,:,:,6] = pot_op[0] # 'C_1P1'
        hf_sms['Vi_sing'][2*j_index,:,:,14] = pot_op[1] # 'D_1P1'
        hf_sms['Vi_sing'][2*j_index+1,:,:,7] = pot_op[2] # 'C_3P1'        
        hf_sms['Vi_sing'][2*j_index+1,:,:,15] = pot_op[3] # 'D_3P1'
        
        hf_sms['Vi_coup'][j_index-1,:num_k,:num_k,3] = pot_op[4] # 'CT_3S1'
        hf_sms['Vi_coup'][j_index-1,:num_k,:num_k,8] = pot_op[5] # 'C_3S1'
        hf_sms['Vi_coup'][j_index-1,:num_k,:num_k,17] = pot_op[6] # 'D_3S1'
        hf_sms['Vi_coup'][j_index-1,:num_k,num_k:2*num_k,9] = pot_op[7] # 'C_e1'
        hf_sms['Vi_coup'][j_index-1,:num_k,num_k:2*num_k,20] = pot_op[8] # 'D_e1'
        hf_sms['Vi_coup'][j_index-1,num_k:2*num_k,:num_k,9] = pot_op[9] # 'C_e1'
        hf_sms['Vi_coup'][j_index-1,num_k:2*num_k,:num_k,20] = pot_op[10] # 'D_e1'
        hf_sms['Vi_coup'][j_index-1,num_k:2*num_k,num_k:2*num_k,18] = pot_op[11] # 'D_3D1'

    elif (j_index == 2):       
        hf_sms['Vi_sing'][2*j_index,:,:,21] = pot_op[0] # 'D_1D2'
        hf_sms['Vi_sing'][2*j_index+1,:,:,22] = pot_op[1] # 'D_3D2'
        
        hf_sms['Vi_coup'][j_index-1,:num_k,:num_k,10] = pot_op[2] # 'C_3P2'
        hf_sms['Vi_coup'][j_index-1,:num_k,:num_k,23] = pot_op[3] # 'D_3P2'
        hf_sms['Vi_coup'][j_index-1,:num_k,num_k:2*num_k,24] = pot_op[4] # 'D_e2'
        hf_sms['Vi_coup'][j_index-1,num_k:2*num_k,:num_k,24] = pot_op[5] # 'D_e2'
        hf_sms['Vi_coup'][j_index-1,num_k:2*num_k,num_k:2*num_k,26] = pot_op[6] # 'E_3F2'

    elif (j_index == 3):        
        hf_sms['Vi_sing'][2*j_index,:,:,27] = pot_op[0] # 'E_1F3'
        hf_sms['Vi_sing'][2*j_index+1,:,:,28] = pot_op[1] # 'E_3F3'
        hf_sms['Vi_coup'][j_index-1,:num_k,:num_k,25] = pot_op[2] # 'D_3D3'

    elif (j_index == 4):
        hf_sms['Vi_coup'][j_index-1,:num_k,:num_k,29] = pot_op[0] # 'E_3F4'
            


In [17]:
hf_sms.close()